In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/solo-destination-and-jaki'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/solo-destination-and-jaki/train_data.csv
/kaggle/input/solo-destination-and-jaki/test_data.csv


In [2]:
# Membaca dataset train
train_data = pd.read_csv('/kaggle/input/solo-destination-and-jaki/train_data.csv', sep=";")

# Membaca dataset test
test_data = pd.read_csv('/kaggle/input/solo-destination-and-jaki/test_data.csv', sep=";")

In [3]:
train_data.dropna()
test_data.dropna()

,content,Sentiment
0,moga manfaat,2
1,sip,2
2,terimakasih anis presiden tolong pj gubernur h...,2
3,sudah install aplikasi fitur jakclm nya tidak,0
4,info kini,2
...,...,...
943,aplikasi bagus bantu terima kasih pemprov dki ...,2
944,halo jaki tolong yang aplikasi jaki sudah habi...,0
945,keren moga dengan jaki atas aaamiin,2
946,ok,2


In [4]:
pip install transformers datasets torch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['content'], train_data['Sentiment'], test_size=0.2, random_state=42
)

In [7]:
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)
test_data['content'] = test_data['content'].astype(str)

In [8]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data['content']), truncation=True, padding=True, max_length=128)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, list(train_labels))
val_dataset = SentimentDataset(val_encodings, list(val_labels))
test_dataset = SentimentDataset(test_encodings, list(test_data['Sentiment']))

In [10]:
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p2', num_labels=3)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='/kaggle/working/results',          # Direktori untuk menyimpan hasil
    evaluation_strategy="steps",    # Evaluasi setiap epoch
    save_strategy="steps",          # Simpan model setiap epoch
    save_steps=1500,  
    load_best_model_at_end=True,    # Memuat model terbaik di akhir
    num_train_epochs=5,             # Jumlah epoch
    per_device_train_batch_size=16, # Batch size untuk training
    per_device_eval_batch_size=16,  # Batch size untuk evaluasi
    logging_dir='./logs',           # Direktori untuk log
    logging_steps=500,               # Frekuensi logging
    save_total_limit=1,             # Simpan hanya model terbaik
    metric_for_best_model="accuracy", # Gunakan akurasi untuk model terbaik
    greater_is_better=True,          # Akurasi lebih tinggi lebih baik
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p2",
    num_labels=3,  # Ganti dengan jumlah label Anda
    ignore_mismatched_sizes=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.495700,0.432401,0.882305,0.882280,0.882469,0.882305
1000,0.183900,0.405775,0.898539,0.898789,0.899539,0.898539
1500,0.095000,0.460069,0.899351,0.898866,0.899001,0.899351


TrainOutput(global_step=1540, training_loss=0.25384129378702736, metrics={'train_runtime': 285.6083, 'train_samples_per_second': 86.202, 'train_steps_per_second': 5.392, 'total_flos': 1328465812829400.0, 'train_loss': 0.25384129378702736, 'epoch': 5.0})

In [13]:
pip install optuna

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [14]:
import optuna
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

# Fungsi untuk training dengan hyperparameter yang diuji
def objective(trial):
    # Mendefinisikan hyperparameter yang akan diuji
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    weight_decay = trial.suggest_uniform('weight_decay', 0.001, 0.1)

    # Menyiapkan TrainingArguments
    training_args = TrainingArguments(
        output_dir='/kaggle/working/results',          # Direktori untuk menyimpan hasil
        evaluation_strategy="steps",     
        save_strategy="steps",
        save_steps=500,
        learning_rate=learning_rate,    # Learning rate
        per_device_train_batch_size=batch_size,  # Ukuran batch
        per_device_eval_batch_size=batch_size,   # Ukuran batch untuk evaluasi
        num_train_epochs=5,       # Jumlah epoch
        weight_decay=weight_decay,              # L2 regularization
        logging_dir='./logs',                   # Direktori untuk menyimpan log
        logging_steps=250,
        load_best_model_at_end=True,            # Simpan model terbaik
        metric_for_best_model='accuracy',      # Metrik untuk memilih model terbaik
        report_to = 'none'
    )

    # Trainer
    trainer = Trainer(
        model=model,                         # Model
        args=training_args,                  # Arguments
        train_dataset=train_dataset,         # Dataset untuk training
        eval_dataset=val_dataset,           # Dataset untuk evaluasi
        compute_metrics=compute_metrics      # Fungsi evaluasi
    )

    # Latih model dan evaluasi
    trainer.train()

    # Evaluasi pada dataset evaluasi
    eval_results = trainer.evaluate()
    return eval_results['eval_accuracy']  # Kembalikan nilai accuracy sebagai hasil

In [15]:
# Membuat study untuk optimasi
study = optuna.create_study(direction='maximize')  # 'maximize' karena kita ingin memaksimalkan akurasi
study.optimize(objective, n_trials=5)  # Menjalankan 5 percobaan

# Menampilkan hasil terbaik
print(f'Best trial: {study.best_trial.params}')

[I 2025-01-03 14:44:43,054] A new study created in memory with name: no-name-ccb28840-e9b0-48ba-905a-f0966d1b8167
<ipython-input-14-3ebfbc8a1665>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-14-3ebfbc8a1665>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.001, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
250,0.749000,0.619495,0.779221,0.778746,0.786802,0.779221
500,0.558800,0.434680,0.841721,0.840164,0.848218,0.841721
750,0.400800,0.510883,0.862825,0.861702,0.867923,0.862825
1000,0.312500,0.365645,0.893669,0.894097,0.894855,0.893669
1250,0.276300,0.432193,0.896916,0.896967,0.897103,0.896916
1500,0.168700,0.443643,0.896916,0.896826,0.897011,0.896916
1750,0.190700,0.436196,0.893669,0.893598,0.893979,0.893669
2000,0.140300,0.488345,0.898539,0.898852,0.900424,0.898539
2250,0.111600,0.573648,0.887175,0.886767,0.892459,0.887175
2500,0.117500,0.521454,0.902597,0.902911,0.904048,0.902597


[I 2025-01-03 14:51:27,411] Trial 0 finished with value: 0.9025974025974026 and parameters: {'batch_size': 8, 'learning_rate': 2.8673946666843204e-05, 'weight_decay': 0.020415441642930344}. Best is trial 0 with value: 0.9025974025974026.
<ipython-input-14-3ebfbc8a1665>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-14-3ebfbc8a1665>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.001, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
250,0.081000,0.515490,0.901786,0.902055,0.902414,0.901786
500,0.064100,0.515106,0.905032,0.905385,0.906425,0.905032
750,0.059900,0.529449,0.905844,0.906186,0.906749,0.905844


[I 2025-01-03 14:55:58,490] Trial 1 finished with value: 0.9050324675324676 and parameters: {'batch_size': 32, 'learning_rate': 1.621539177964685e-05, 'weight_decay': 0.08500836041371965}. Best is trial 1 with value: 0.9050324675324676.
<ipython-input-14-3ebfbc8a1665>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-14-3ebfbc8a1665>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.001, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.

...

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
250,0.065700,0.624079,0.897727,0.897563,0.898178,0.897727
500,0.067100,0.653943,0.898539,0.898279,0.899345,0.898539
750,0.057700,0.670192,0.892857,0.891901,0.892449,0.892857
1000,0.055500,0.652120,0.897727,0.897620,0.898127,0.897727
1250,0.062400,0.659653,0.899351,0.899207,0.899794,0.899351
1500,0.057000,0.659140,0.900162,0.900049,0.900766,0.900162


[I 2025-01-03 15:06:12,530] Trial 3 finished with value: 0.9001623376623377 and parameters: {'batch_size': 16, 'learning_rate': 1.0811741862385977e-05, 'weight_decay': 0.07535661077959584}. Best is trial 1 with value: 0.9050324675324676.
<ipython-input-14-3ebfbc8a1665>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
<ipython-input-14-3ebfbc8a1665>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.001, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
250,0.161000,0.741565,0.859578,0.858516,0.863291,0.859578
500,0.112300,0.686416,0.885552,0.885462,0.885420,0.885552
750,0.090400,0.642935,0.891234,0.890398,0.890515,0.891234
1000,0.074300,0.678153,0.888799,0.888953,0.889343,0.888799
1250,0.068700,0.666886,0.890422,0.890495,0.891189,0.890422
1500,0.058400,0.681487,0.891234,0.891310,0.891451,0.891234


[I 2025-01-03 15:11:19,094] Trial 4 finished with value: 0.8912337662337663 and parameters: {'batch_size': 16, 'learning_rate': 4.2406857680850346e-05, 'weight_decay': 0.07100409306087847}. Best is trial 1 with value: 0.9050324675324676.


Best trial: {'batch_size': 32, 'learning_rate': 1.621539177964685e-05, 'weight_decay': 0.08500836041371965}


In [16]:
# Menampilkan hasil terbaik
print(f'Best trial: {study.best_trial.params}')

Best trial: {'batch_size': 32, 'learning_rate': 1.621539177964685e-05, 'weight_decay': 0.08500836041371965}


In [18]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',          # Direktori untuk menyimpan hasil
    evaluation_strategy="steps",    # Evaluasi setiap epoch
    save_strategy="steps",          # Simpan model setiap epoch
    save_steps=1500,
    learning_rate=1.621539177964685e-05,
    num_train_epochs=20,             # Jumlah epoch
    per_device_train_batch_size=32, # Batch size untuk training
    per_device_eval_batch_size=32,  # Batch size untuk evaluasi
    logging_dir='./logs',           # Direktori untuk log
    logging_steps=250,
    weight_decay=0.08500836041371965,# Frekuensi logging
    save_total_limit=1,             # Simpan hanya model terbaik
    metric_for_best_model="accuracy", # Gunakan akurasi untuk model terbaik
    greater_is_better=True,          # Akurasi lebih tinggi lebih baik
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
250,0.057300,0.709990,0.893669,0.893686,0.894236,0.893669
500,0.058600,0.715056,0.895292,0.895597,0.895992,0.895292
750,0.060700,0.768592,0.887987,0.887871,0.887862,0.887987
1000,0.059700,0.775603,0.889610,0.889614,0.889619,0.889610
1250,0.058500,0.864150,0.882305,0.881758,0.881818,0.882305
1500,0.057100,0.791324,0.891234,0.891289,0.891381,0.891234
1750,0.057300,0.760205,0.896916,0.897060,0.897529,0.896916
2000,0.057000,0.768801,0.895292,0.895439,0.895910,0.895292
2250,0.055100,0.770057,0.895292,0.895439,0.895910,0.895292
2500,0.058400,0.777691,0.895292,0.895397,0.895797,0.895292


TrainOutput(global_step=3080, training_loss=0.05738637617656163, metrics={'train_runtime': 1068.148, 'train_samples_per_second': 92.197, 'train_steps_per_second': 2.883, 'total_flos': 5313863251317600.0, 'train_loss': 0.05738637617656163, 'epoch': 20.0})

In [19]:
from sklearn.metrics import classification_report

# Evaluasi model pada dataset validasi
eval_results = trainer.evaluate()

# Cetak metrik yang dihitung selama evaluasi
print("Evaluation Metrics:")
print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"F1-score: {eval_results['eval_f1']}")
print(f"Precision: {eval_results['eval_precision']}")
print(f"Recall: {eval_results['eval_recall']}")

# Prediksi pada dataset validasi untuk membuat laporan klasifikasi
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)
true_labels = predictions.label_ids

Evaluation Metrics:
Accuracy: 0.8952922077922078
F1-score: 0.8953664453776438
Precision: 0.8956700661254663
Recall: 0.8952922077922078


In [20]:
# Membaca dataset test
test_data = pd.read_csv('/kaggle/input/solo-destination-and-jaki/test_data.csv', sep=";")

# Memisahkan kolom 'content' dan 'Sentiment'
content_column = test_data['content']  # Kolom teks
sentiment_column = test_data['Sentiment']  # Kolom label

# Menampilkan hasil untuk verifikasi
print("Kolom Content:")
print(content_column.head())  # Menampilkan 5 baris pertama kolom 'content'

print("\nKolom Sentiment:")
print(sentiment_column.head())  # Menampilkan 5 baris pertama kolom 'Sentiment'

Kolom Content:
0                                         moga manfaat
1                                                  sip
2    terimakasih anis presiden tolong pj gubernur h...
3        sudah install aplikasi fitur jakclm nya tidak
4                                            info kini
Name: content, dtype: object

Kolom Sentiment:
0    2
1    2
2    2
3    0
4    2
Name: Sentiment, dtype: int64


In [21]:
# Periksa nilai NaN dalam kolom content_column
print(content_column.isnull().sum())

15


In [22]:
# Hapus baris dengan nilai NaN
content_column = content_column.dropna()

# Pastikan semua data adalah string
content_column = content_column.astype(str)

In [23]:
test_encodings = tokenizer(list(content_column), truncation=True, padding=True, max_length=128)

In [24]:
# Membuat Dataset untuk Prediksi
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

test_dataset = SentimentDataset(test_encodings)

In [26]:
# Memastikan tidak ada data kosong pada kolom content
test_data = test_data[test_data['content'].notna()].reset_index(drop=True)

In [27]:
# Tokenisasi ulang
test_encodings = tokenizer(list(test_data['content']), truncation=True, padding=True, max_length=128)

In [28]:
# Prediksi ulang
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Tambahkan hasil prediksi ke dataset asli
test_data['Predicted_Sentiment'] = predicted_labels

In [29]:
assert len(test_data) == len(predicted_labels), "Jumlah prediksi tidak sesuai dengan jumlah data!"
print(test_data[['content', 'Sentiment', 'Predicted_Sentiment']].head())

                                             content  Sentiment  \
0                                       moga manfaat          2   
1                                                sip          2   
2  terimakasih anis presiden tolong pj gubernur h...          2   
3      sudah install aplikasi fitur jakclm nya tidak          0   
4                                          info kini          2   

   Predicted_Sentiment  
0                    2  
1                    2  
2                    2  
3                    0  
4                    2  


In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Pastikan kolom Predicted_Sentiment ada di test_data
if 'Predicted_Sentiment' not in test_data.columns:
    raise ValueError("Kolom 'Predicted_Sentiment' tidak ditemukan di test_data. Pastikan prediksi telah dilakukan.")

# Ambil label sebenarnya dan label prediksi
true_labels = test_data['Sentiment']
predicted_labels = test_data['Predicted_Sentiment']

# Hitung metrik evaluasi
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')  # Menggunakan rata-rata berbobot
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Menampilkan hasil
print(f"Akurasi: {accuracy:.4f}")
print(f"Presisi: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Menampilkan laporan klasifikasi
print("\nLaporan Klasifikasi:")
print(classification_report(true_labels, predicted_labels))

Akurasi: 0.7985
Presisi: 0.7871
Recall: 0.7985
F1-Score: 0.7924

Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       372
           1       0.18      0.13      0.15        61
           2       0.86      0.87      0.87       500

    accuracy                           0.80       933
   macro avg       0.61      0.60      0.61       933
weighted avg       0.79      0.80      0.79       933

